In [4]:
# import our logic analyzer data

import pandas as pd
import numpy as np
from pandas import Series, DataFrame

df = pd.read_csv('bfin-known.csv')
cmds = df[' Decoded Protocol Result']
cmds

0       Run-Test/Idle
1      Select-DR-Scan
2      Select-IR-Scan
3          Capture-IR
4            Shift-IR
            ...      
539          Shift-DR
540       0x000000007
541       0x78000000A
542          Exit1-DR
543         Update-DR
Name:  Decoded Protocol Result, Length: 544, dtype: object

In [7]:
import xml.etree.ElementTree as ET

# get registers to cross-reference
tree = ET.parse('ADSP-BF706-registers.xml')
root = tree.getroot()
regdefs = root[2]

fullregs = []
for r in regdefs.findall('register[@read-address]'):
    atts = r.attrib
    fullregs.append({'name' : atts['name'], 'din' : atts['read-address']})

fullregs.append({'name' : 'EMUIR', 'din' : '0x80001008'})
fullregs.append({'name' : 'EMUDAT', 'din' : '0x8000100C'})
fullregs.append({'name' : 'DSCR', 'din' : '0x80001018'}) #this is not exact match to DSCR
    
# I cannot find any reference to what these registers do anywhere
fullregs.append({'name' : 'MYSTERY0 (0x80001000)', 'din' : '0x80001000'})
fullregs.append({'name' : 'MYSTERY1C (0x8000101C)', 'din' : '0x8000101C'})
    
regs = DataFrame(fullregs)
regs

,name,din
0,L1DM_SRAM_BASE_ADDR,0x1FC00000
1,L1DM_DCTL,0x1FC00004
2,L1DM_DSTAT,0x1FC00008
3,L1DM_DCPLB_FAULT_ADDR,0x1FC0000C
4,L1DM_DCPLB_DFLT,0x1FC00010
...,...,...
3115,EMUIR,0x80001008
3116,EMUDAT,0x8000100C
3117,DSCR,0x80001018
3118,MYSTERY0 (0x80001000),0x80001000


In [9]:
# build Cross Trigger Interface register list
prefix = "SYSCTI_"
offset = 0x2001A000
def toInst(x):
    x['name'] = prefix + x['name']
    x['din'] = '0x' + format(int(x['addr'], 0) + offset, '08X')
    return x

cti = pd.read_csv('cti.csv')

syscti = cti.copy()
syscti = syscti.apply(toInst, axis=1)

prefix = "PROCCTI_"
offset = 0x20013000
proccti = cti.copy()
proccti = cti.apply(toInst, axis=1)

fullcti = proccti.append(syscti).reset_index()[['name', 'din']]

,name,din
0,PROCCTI_CTICONTROL,0x20013000
1,PROCCTI_CTISTATUS,0x20013004
2,PROCCTI_CTILOCK,0x20013008
3,PROCCTI_CTIPROTECTION,0x2001300C
4,PROCCTI_CTIINTACK,0x20013010
...,...,...
69,SYSCTI_CTIPERIPHID3,0x2001AFEC
70,SYSCTI_CTIPCELLID0,0x2001AFF0
71,SYSCTI_CTIPCELLID1,0x2001AFF4
72,SYSCTI_CTIPCELLID2,0x2001AFF8


In [15]:
# define the processor registers so that we can label them in our pseudo-disassembly

T_REG_R = 0x00
T_REG_P = 0x10
T_REG_I = 0x20
T_REG_B = 0x30
T_REG_L = 0x34
T_REG_M = 0x24
T_REG_A = 0x40

regnos = {0x40 : 'A0x', 0x41 : 'A0w', 0x42 : 'A1x', 0x43 : 'A1w', \
          0x46 : 'ATSTAT', 0x47 : 'RETS', 0x60 : 'LC0', 0x61 : 'LT0', \
         0x62 : 'LB0', 0x61 : 'LC1', 0x62 : 'LT1', 0x63 : 'LB1', \
         0x64 : 'CYCLES', 0x65 : 'CYCLES2', 0x70 : 'USP', 0x71 : 'SEQSTAT', \
          0x72 : 'SYSCFG', 0x73 : 'RETI', 0x74 : 'RETX', 0x75 : 'RETN', 0x76 : 'RETE', 0x77 : 'EMUDAT'}

for i in range(8):
    regnos[T_REG_R + i] = 'R' + str(i)
    
for i in range(6):
    regnos[T_REG_P + i] = 'P' + str(i)
regnos[0x16] = 'SP'
regnos[0x17] = 'FP'

for i in range(4):
    regnos[T_REG_I + i] = 'I' + str(i)
    
for i in range(4):
    regnos[T_REG_M + i] = 'M' + str(i)
    
for i in range(4):
    regnos[T_REG_B + i] = 'B' + str(i)
    
for i in range(4):
    regnos[T_REG_L + i] = 'L' + str(i)
    
def parse_ldst(i):
    reg = i & 0x7
    ptr = (i >> 3) & 0x7
    post_dec = (i >> 7) & 0x3
    w = (i >> 9) & 0x1
    sz = (i >> 10) & 0xF
    op = ""
    if post_dec == 0:
        op = "++"
    elif post_dec == 1:
        op = "--"
    if w:
        return  "[" + regnos[T_REG_P + ptr] + op + "] = " + regnos[T_REG_R + reg]
    else:
        return regnos[T_REG_R + reg] + " = [" + regnos[T_REG_P + ptr] + op + "]"

def parse_move(i):
    src = i & 0x7
    dst = (i >> 3) & 0x7
    g_src = ((i >> 6) & 0x7) << 4
    g_dst = ((i >> 9) & 0x7) << 4
    return regnos[g_dst + dst] + " = " + regnos[g_src + src]
    
def parse_emuir(ins):
    i = int(ins, 0) >> 16
    if i & 0x3000 == 0x3000:
        return parse_move(i)
    elif i & 0x9000 == 0x9000:
        return parse_ldst(i)
    else:
        return "unknown (" + ins + ")"

In [16]:
# parse out the JTAG state machine

state = []
tdi = []
tdo = []

for ix, c in enumerate(cmds):
    last = None
    if ix > 0:
        last = cmds[ix - 1]
        
    if c[:2] == "0x" and last[:2] != "0x":
        state.append(last)
        tdi.append(c)
        tdo.append(cmds[ix + 1])
        
jtag_cmds = []
for c in tdi:
    if c == "0x1FB":
        jtag_cmds.append("APACC")
    elif c == "0x1FA":
        jtag_cmds.append("DPACC")
    elif len(c) == 11:
        jtag_cmds.append("data")
    else:
        jtag_cmds.append("unknown")
        
response = []
dout = []

for c in tdo:
    try:
        d = int(c, 0) & 0x7
        if d == 0b010:
            response.append("OK/FAULT")
        elif d == 0b001:
            response.append("WAIT")
        else:
            response.append("UNKNOWN")

        dout.append('0x' + format(int(c, 0) >> 3, '08X'))
    except:
        response.append("ERROR")
        dout.append("error: " + c)

a = []
rnw = []
din = []
for c in tdi:
    try:
        i = int(c, 0)
        if i & 0x01 == 1:
            rnw.append("R")
        else:
            rnw.append("W")

        a.append(format((i & 0x6) << 1, '01x'))
        din.append('0x' + format(i >> 3, '08X'))
    except:
        rnw.append("error")
        a.append("error")
        din.append("error: " + c)

#no response and state for now
df = DataFrame({'jtag_cmds': jtag_cmds, \
                'tdi' : tdi, 'tdo' : tdo, \
                'A' : a, 'RnW' : rnw, \
               'din' : din, 'dout' : dout})

df = df.join(regs.set_index('din'), on='din')
df = df.merge(fullcti, on='din', how='left')
df['name'] = pd.concat([df['name_x'].dropna(), df['name_y'].dropna()]).reindex_like(df)
df = df.drop(['name_x', 'name_y'], axis=1)


newdf = DataFrame()
#get rid of the dpacc and apacc rows
for index, row in df.iterrows():
    if row['jtag_cmds'] == 'data' and ( df.iloc[index - 1]['jtag_cmds'] == 'DPACC' or df.iloc[index - 1]['jtag_cmds'] == 'APACC' ):
        row['jtag_cmds'] = df.iloc[index - 1]['jtag_cmds']
        newdf = newdf.append(row)
    elif row['jtag_cmds'] == "unknown":
        newdf = newdf.append(row)
        
df = newdf.reset_index(drop=True)

df = df.fillna(value='-')

print(str(len(df)) + " total rows")
df.head()

32 total rows


,A,RnW,din,dout,jtag_cmds,name,tdi,tdo
0,4,W,0x8000100C,0x00000000,APACC,EMUDAT,0x400008062,0x000000002
1,c,W,0x1FC01004,0x00000203,APACC,L1IM_ICTL,0x0FE008026,0x00000101A
2,4,W,0x80001008,0x00000203,APACC,EMUIR,0x400008042,0x00000101A
3,c,W,0x33C70000,0x00000203,APACC,-,0x19E380006,0x00000101A
4,4,W,0x80001008,0x00000203,APACC,EMUIR,0x400008042,0x00000101A


In [17]:
#rephrase based on what we know now

ports = ['HB-AP', 'PB-AP', 'JTAG-AP']

APSEL = 'PB-AP' #currently selected AP
line = ""
insns = []
curAddr = '0x00'
for ix, row in df.iterrows():
    line = "<"+str(ix)+">  "
    if row['jtag_cmds'] == 'DPACC':
        if row['RnW'] == 'W' and row['A'] == '8':
            try:
                APSEL = ports[int(row['din'], 0) >> 24]
            except:
                APSEL = 'UNKNOWN AP (' + str(int(row['din'], 0) >> 24)  + ')'
            line = line + "select " + APSEL
            insns.append(line)
            line = ""
    if row['jtag_cmds'] == 'APACC':
        op = ""
        if row['RnW'] == 'W':
            op = "write "
        else:
            op = "read "
                
        if row['RnW'] == 'W' and row['A'] == '4':
            if row['name'] != '-':
                curAddr = row['name']
            else:
                curAddr = row['din']
        elif row['A'] == 'c':
            data = row['din']
            if row['RnW'] == 'R' and len(df) > ix + 1:
                data = df.iloc[ix + 1]['dout']
            elif row['name'] != '-':
                data = row['name']
                
            if curAddr == 'EMUIR':
                try:
                    line = line + parse_emuir(row['din'])
                except:
                    line = line + row['din']
            else:
                line = line + op + data + " at " + curAddr + " ("+ APSEL + ")"
            insns.append(line)
            line = ""
        elif row['A'] == '0':
            line = line + op + "CSW: " +row['din'] + " ("+ APSEL + ")"
            insns.append(line)
            line = ""
    if row['tdo'] == '0x78000000A':
        line = ""
        insns.append(line)
        line = ""
    if row['jtag_cmds'] == 'unknown':
        line = row['din']
        insns.append(line)
        line = ""

print("\n".join(insns))

<1>  write L1IM_ICTL at EMUDAT (PB-AP)
<3>  P0 = EMUDAT
<5>  R0 = [P0]
<7>  EMUDAT = R0
<9>  read 0x00000207 at EMUDAT (PB-AP)

<13>  write L1DM_DCTL at EMUDAT (PB-AP)
<15>  P0 = EMUDAT
<17>  R0 = [P0]
<19>  EMUDAT = R0
<21>  read 0x00000203 at EMUDAT (PB-AP)

<24>  select HB-AP

<28>  write CSW: 0x03800052 (HB-AP)
<29>  read 0x2280A0CB at TAPC0_IDCODE (HB-AP)

